# Създайте регресионен модел: линейни и полиномиални регресионни модели


## Линейна и полиномиална регресия за ценообразуване на тикви - Урок 3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Инфографика от Дасани Мадипали</figcaption>


#### Въведение

Досега разгледахте какво представлява регресията с примерни данни, събрани от набора с данни за ценообразуване на тикви, който ще използваме през целия този урок. Също така го визуализирахте с помощта на `ggplot2`. 💪

Сега сте готови да се задълбочите в регресията за машинно обучение. В този урок ще научите повече за два типа регресия: *основна линейна регресия* и *полиномиална регресия*, както и за част от математиката, която стои зад тези техники.

> В рамките на тази учебна програма предполагаме минимални познания по математика и се стремим да я направим достъпна за студенти от други области, така че следете за бележки, 🧮 подсказки, диаграми и други учебни инструменти, които да помогнат за разбирането.

#### Подготовка

Като напомняне, зареждате тези данни, за да задавате въпроси към тях.

-   Кога е най-доброто време за купуване на тикви?

-   Каква цена мога да очаквам за кутия с миниатюрни тикви?

-   Трябва ли да ги купувам в кошници с половин бушел или в кутии с 1 1/9 бушел? Нека продължим да изследваме тези данни.

В предишния урок създадохте `tibble` (съвременна преосмислена версия на рамка за данни) и го попълнихте с част от оригиналния набор от данни, стандартизирайки цените по бушел. С това обаче успяхте да съберете само около 400 точки данни и само за есенните месеци. Може би можем да получим малко повече подробности за естеството на данните, като ги почистим повече? Ще видим... 🕵️‍♀️

За тази задача ще ни трябват следните пакети:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) е [колекция от R пакети](https://www.tidyverse.org/packages), създадена да направи науката за данни по-бърза, лесна и забавна!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) е рамка, представляваща [колекция от пакети](https://www.tidymodels.org/packages/) за моделиране и машинно обучение.

-   `janitor`: [janitor](https://github.com/sfirke/janitor) предоставя прости инструменти за изследване и почистване на "мръсни" данни.

-   `corrplot`: [corrplot](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) предлага визуален инструмент за изследване на матрици на корелация, който поддържа автоматично пренареждане на променливите, за да помогне за откриване на скрити модели между тях.

Можете да ги инсталирате със следната команда:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

Скриптът по-долу проверява дали имате необходимите пакети за завършване на този модул и ги инсталира, ако липсват.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

Ще заредим тези страхотни пакети и ще ги направим достъпни в текущата R сесия. (Това е само за илюстрация, `pacman::p_load()` вече го направи за вас)

## 1. Линия на линейна регресия

Както научихте в Урок 1, целта на упражнението по линейна регресия е да начертаете *линия* *на* *най-добро съответствие*, за да:

-   **Покажете връзките между променливите**. Демонстрирайте връзката между променливите.

-   **Направите прогнози**. Направете точни прогнози за това къде нова точка от данни би попаднала спрямо тази линия.

За да начертаем този тип линия, използваме статистическа техника, наречена **Регресия на най-малките квадрати**. Терминът `най-малки квадрати` означава, че всички точки от данни около регресионната линия се повдигат на квадрат и след това се сумират. Идеално, тази крайна сума трябва да бъде възможно най-малка, защото искаме нисък брой грешки, или `най-малки квадрати`. Следователно, линията на най-добро съответствие е линията, която ни дава най-ниската стойност за сумата на квадратите на грешките - оттук и името *регресия на най-малките квадрати*.

Правим това, защото искаме да моделираме линия, която има най-малко кумулативно разстояние от всички наши точки от данни. Освен това повдигаме на квадрат стойностите преди да ги сумираме, защото се интересуваме от тяхната величина, а не от посоката им.

> **🧮 Покажете ми математиката**
>
> Тази линия, наречена *линия на най-добро съответствие*, може да бъде изразена чрез [уравнение](https://en.wikipedia.org/wiki/Simple_linear_regression):
>
>     Y = a + bX
>
> `X` е '`обяснителна променлива` или `предиктор`'. `Y` е '`зависима променлива` или `резултат`'. Наклонът на линията е `b`, а `a` е пресечната точка с оста Y, която се отнася до стойността на `Y`, когато `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "наклон = $y/x$")
    Инфографика от Jen Looper
>
> Първо, изчислете наклона `b`.
>
> С други думи, и като се позоваваме на оригиналния въпрос за данните за тиквите: "предскажете цената на тиква на бушел по месеци", `X` би се отнасял до цената, а `Y` би се отнасял до месеца на продажба.
>
> ![](../../../../../../2-Regression/3-Linear/solution/images/calculation.png)
    Инфографика от Jen Looper
> 
> Изчислете стойността на Y. Ако плащате около \$4, значи е април!
>
> Математиката, която изчислява линията, трябва да демонстрира наклона на линията, който също зависи от пресечната точка, или къде се намира `Y`, когато `X = 0`.
>
> Можете да наблюдавате метода за изчисление на тези стойности на уебсайта [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html). Също така посетете [този калкулатор за най-малки квадрати](https://www.mathsisfun.com/data/least-squares-calculator.html), за да видите как стойностите на числата влияят на линията.

Не е толкова страшно, нали? 🤓

#### Корелация

Още един термин, който трябва да разберете, е **Коефициентът на корелация** между дадените X и Y променливи. С помощта на диаграма с разпръснати точки можете бързо да визуализирате този коефициент. Диаграма с точки, разположени в подредена линия, има висока корелация, но диаграма с точки, разпръснати навсякъде между X и Y, има ниска корелация.

Добър модел на линейна регресия ще бъде този, който има висок (по-близо до 1, отколкото до 0) коефициент на корелация, използвайки метода на регресия на най-малките квадрати с линия на регресия.


## **2. Танц с данните: създаване на датафрейм за моделиране**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>Илюстрация от @allison_horst</figcaption>


<!--![Илюстрация от \@allison_horst](../../../../../../2-Regression/3-Linear/images/janitor.jpg){width="700"}-->


Заредете необходимите библиотеки и набора от данни. Преобразувайте данните в дата фрейм, съдържащ подмножество от данните:

-   Вземете само тикви, оценени на цена за бушел

-   Преобразувайте датата в месец

-   Изчислете цената като средна стойност между високата и ниската цена

-   Преобразувайте цената, за да отразява ценообразуването според количеството в бушел

> Покрихме тези стъпки в [предишния урок](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb).


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

В духа на чистото приключение, нека изследваме [`janitor package`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor), който предоставя прости функции за изследване и почистване на замърсени данни. Например, нека разгледаме имената на колоните за нашите данни:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 Можем да се справим по-добре. Нека направим тези имена на колони `friendR`, като ги преобразуваме към конвенцията [snake_case](https://en.wikipedia.org/wiki/Snake_case), използвайки `janitor::clean_names`. За да научите повече за тази функция: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

Много tidyR 🧹! Сега, танц с данните, използвайки `dplyr`, както в предишния урок! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Добра работа!👌 Сега разполагате с чист и подреден набор от данни, върху който можете да изградите новия си регресионен модел!

Какво ще кажете за диаграма на разсейване?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


Диаграмата с разпръснати точки ни напомня, че разполагаме с данни само за месеците от август до декември. Вероятно ще ни трябват повече данни, за да можем да правим изводи по линеен начин.

Нека отново разгледаме нашите данни за моделиране:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Какво би станало, ако искахме да предвидим `цената` на тиква въз основа на колоните `град` или `опаковка`, които са от тип символен? Или още по-просто, как бихме могли да намерим корелацията (която изисква и двата входа да бъдат числови) между, например, `опаковка` и `цена`? 🤷🤷

Моделите за машинно обучение работят най-добре с числови характеристики, а не с текстови стойности, така че обикновено трябва да преобразувате категориалните характеристики в числови представяния.

Това означава, че трябва да намерим начин да преформатираме нашите предиктори, за да ги направим по-лесни за използване от модела, процес, известен като `инженеринг на характеристики`.


## 3. Предварителна обработка на данни за моделиране с recipes 👩‍🍳👨‍🍳

Дейностите, които преформатират стойностите на предикторите, за да ги направят по-лесни за ефективно използване от модела, се наричат `feature engineering` (инженеринг на характеристики).

Различните модели имат различни изисквания за предварителна обработка. Например, методът на най-малките квадрати изисква `кодиране на категорийни променливи`, като месец, сорт и име на град. Това включва просто `превръщане` на колона с `категорийни стойности` в една или повече `числови колони`, които заместват оригиналната.

Например, да предположим, че вашите данни включват следната категорийна характеристика:

|  град   |
|:-------:|
| Денвър  |
| Найроби |
|  Токио  |

Можете да приложите *ординално кодиране*, за да замените всяка категория с уникална целочислена стойност, ето така:

| град |
|:----:|
|  0   |
|  1   |
|  2   |

И точно това ще направим с нашите данни!

В тази секция ще разгледаме още един невероятен пакет от Tidymodels: [recipes](https://tidymodels.github.io/recipes/) - който е създаден, за да ви помогне да подготвите данните си **преди** обучението на модела. В основата си, рецептата е обект, който определя какви стъпки трябва да бъдат приложени към даден набор от данни, за да бъде готов за моделиране.

Сега, нека създадем рецепта, която подготвя нашите данни за моделиране, като заменя всички наблюдения в колоните на предикторите с уникални цели числа:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

Страхотно! 👏 Току-що създадохме първата си рецепта, която определя резултат (цена) и съответните предиктори, като всички колони с предиктори трябва да бъдат кодирани като набор от цели числа 🙌! Нека бързо да я разгледаме:

-   Извикването на `recipe()` с формула указва на рецептата *ролите* на променливите, използвайки данните от `new_pumpkins` като референция. Например, колоната `price` е определена с роля `outcome`, докато останалите колони са определени с роля `predictor`.

-   `step_integer(all_predictors(), zero_based = TRUE)` указва, че всички предиктори трябва да бъдат преобразувани в набор от цели числа, като номерирането започва от 0.

Сигурни сме, че може би си мислите: "Това е толкова готино!! Но какво ако трябва да се уверя, че рецептите правят точно това, което очаквам? 🤔"

Това е страхотна мисъл! Виждате ли, след като рецептата е дефинирана, можете да изчислите параметрите, необходими за действителната предварителна обработка на данните, и след това да извлечете обработените данни. Обикновено не е необходимо да правите това, когато използвате Tidymodels (ще видим нормалната конвенция след малко -> `workflows`), но може да бъде полезно, когато искате да направите проверка за потвърждение, че рецептите правят това, което очаквате.

За това ще ви трябват още два глагола: `prep()` и `bake()`, а както винаги, нашите малки приятели от R, създадени от [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations), ще ви помогнат да разберете това по-добре!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Илюстрация от @allison_horst</figcaption>


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): изчислява необходимите параметри от тренировъчен набор, които по-късно могат да бъдат приложени към други набори от данни. Например, за дадена колона с предиктори, кое наблюдение ще бъде присвоено като цяло число 0, 1, 2 и т.н.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): взема предварително обработена рецепта и прилага операциите към всеки набор от данни.

С това казано, нека подготвим и приложим нашите рецепти, за да потвърдим, че под повърхността колоните с предиктори първо ще бъдат кодирани, преди моделът да бъде обучен.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

Ура! 🥳 Обработените данни `baked_pumpkins` имат всички свои предиктори кодирани, което потвърждава, че стъпките за предварителна обработка, дефинирани като наша рецепта, ще работят както се очаква. Това прави данните по-трудни за четене, но много по-разбираеми за Tidymodels! Отделете малко време, за да разберете коя наблюдавана стойност е била съответно преобразувана в цяло число.

Също така си струва да се спомене, че `baked_pumpkins` е дата фрейм, върху който можем да извършваме изчисления.

Например, нека се опитаме да намерим добра корелация между две точки от вашите данни, за да изградим потенциално добър предсказателен модел. Ще използваме функцията `cor()` за това. Напишете `?cor()`, за да научите повече за функцията.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


Оказва се, че има само слаба връзка между Град и Цена. Въпреки това, има малко по-добра връзка между Пакет и неговата Цена. Това има смисъл, нали? Обикновено, колкото по-голяма е кутията с продукти, толкова по-висока е цената.

Докато сме на тази тема, нека също опитаме да визуализираме матрица на корелация за всички колони, използвайки пакета `corrplot`.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 Много по-добре.

Един добър въпрос, който можем да зададем за тези данни, е: '`Каква цена мога да очаквам за даден пакет тикви?`' Нека започнем веднага!

> Note: Когато **`bake()`** на предварително подготвената рецепта **`pumpkins_prep`** с **`new_data = NULL`**, извличате обработените (т.е. кодирани) тренировъчни данни. Ако имате друг набор от данни, например тестов набор, и искате да видите как рецептата би го обработила, просто изпечете **`pumpkins_prep`** с **`new_data = test_set`**

## 4. Създаване на модел за линейна регресия

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Инфографика от Дасани Мадипали</figcaption>


<!--![Инфографика от Дасани Мадипали](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Сега, след като създадохме рецепта и потвърдихме, че данните ще бъдат предварително обработени правилно, нека изградим регресионен модел, за да отговорим на въпроса: `Каква цена мога да очаквам за даден пакет тикви?`

#### Обучение на линеен регресионен модел с помощта на тренировъчния набор

Както вероятно вече сте се досетили, колоната *price* е променливата `резултат`, докато колоната *package* е променливата `предиктор`.

За да направим това, първо ще разделим данните така, че 80% да отидат в тренировъчния набор, а 20% в тестовия набор. След това ще дефинираме рецепта, която ще кодира колоната предиктор в набор от цели числа, и ще създадем спецификация на модела. Няма да подготвяме и изпичаме рецептата, тъй като вече знаем, че тя ще обработи данните според очакванията.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

Добра работа! Сега, след като имаме рецепта и спецификация на модела, трябва да намерим начин да ги обединим в един обект, който първо ще обработи данните (prep+bake зад кулисите), ще обучи модела върху предварително обработените данни и също така ще позволи потенциални дейности по последваща обработка. Как ти звучи това за спокойствие на ума!🤩

В Tidymodels този удобен обект се нарича [`workflow`](https://workflows.tidymodels.org/) и удобно съхранява твоите компоненти за моделиране! Това е, което бихме нарекли *потоци* в *Python*.

Така че, нека обединим всичко в един workflow!📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

👌 В допълнение, работният процес може да бъде настроен/обучен по почти същия начин, както един модел.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

От изхода на модела можем да видим коефициентите, научени по време на обучението. Те представляват коефициентите на линията на най-добро съответствие, която ни дава най-ниската обща грешка между действителната и предсказаната променлива.

#### Оценка на представянето на модела с помощта на тестовия набор

Време е да видим как се е справил моделът 📏! Как да направим това?

Сега, когато сме обучили модела, можем да го използваме, за да направим прогнози за `test_set`, използвайки `parsnip::predict()`. След това можем да сравним тези прогнози с действителните стойности на етикетите, за да оценим колко добре (или не!) работи моделът.

Нека започнем с правенето на прогнози за тестовия набор, след което да свържем колоните към тестовия набор.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

Да, току-що обучихте модел и го използвахте, за да направите прогнози! 🔮 Добър ли е? Нека оценим представянето на модела!

В Tidymodels правим това с помощта на `yardstick::metrics()`! За линейна регресия нека се фокусираме върху следните метрики:

-   `Root Mean Square Error (RMSE)`: Квадратният корен на [MSE](https://en.wikipedia.org/wiki/Mean_squared_error). Това дава абсолютна метрика в същата единица като етикета (в този случай, цената на тиквата). Колкото по-малка е стойността, толкова по-добър е моделът (в опростен смисъл, тя представлява средната стойност, с която прогнозите грешат спрямо реалните цени!).

-   `Coefficient of Determination (обикновено известен като R-squared или R2)`: Относителна метрика, при която колкото по-висока е стойността, толкова по-добре моделът пасва на данните. По същество тази метрика показва колко от вариацията между прогнозираните и реалните стойности на етикетите моделът успява да обясни.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

Там отива представянето на модела. Нека видим дали можем да получим по-добра индикация, като визуализираме разпръснат график на пакета и цената, след което използваме направените прогнози, за да наложим линия на най-добро съответствие.

Това означава, че ще трябва да подготвим и обработим тестовия набор, за да кодираме колоната за пакета, след което да я свържем с прогнозите, направени от нашия модел.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


Чудесно! Както виждате, линейният регресионен модел не успява добре да обобщи връзката между пакета и съответната му цена.

🎃 Поздравления, току-що създадохте модел, който може да помогне за предсказване на цената на няколко разновидности на тикви. Вашата празнична тиквена градина ще бъде красива. Но вероятно можете да създадете по-добър модел!

## 5. Създайте полиномиален регресионен модел

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Инфографика от Дасани Мадипали</figcaption>


<!--![Инфографика от Дасани Мадипали](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Понякога нашите данни може да нямат линейна връзка, но все пак искаме да прогнозираме резултат. Полиномиалната регресия може да ни помогне да правим прогнози за по-сложни нелинейни връзки.

Вземете например връзката между опаковката и цената в нашия набор от данни за тикви. Докато понякога има линейна връзка между променливите - колкото по-голяма е тиквата по обем, толкова по-висока е цената - понякога тези връзки не могат да бъдат изобразени като равнина или права линия.

> ✅ Ето [някои допълнителни примери](https://online.stat.psu.edu/stat501/lesson/9/9.8) за данни, които могат да използват полиномиална регресия
>
> Погледнете отново връзката между сорта и цената в предишния график. Изглежда ли този разпръснат график като нещо, което задължително трябва да бъде анализирано с права линия? Може би не. В този случай можете да опитате полиномиална регресия.
>
> ✅ Полиномите са математически изрази, които могат да се състоят от една или повече променливи и коефициенти

#### Обучение на модел за полиномиална регресия с помощта на тренировъчния набор

Полиномиалната регресия създава *извита линия*, за да пасне по-добре на нелинейните данни.

Нека видим дали полиномиалният модел ще се представи по-добре при правене на прогнози. Ще следваме подобна процедура, както направихме преди:

-   Създайте рецепта, която определя стъпките за предварителна обработка, които трябва да бъдат изпълнени върху нашите данни, за да ги подготвим за моделиране, например: кодиране на предиктори и изчисляване на полиноми от степен *n*

-   Изградете спецификация на модела

-   Комбинирайте рецептата и спецификацията на модела в работен поток

-   Създайте модел чрез напасване на работния поток

-   Оценете колко добре се представя моделът върху тестовите данни

Да започваме!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Оценка на производителността на модела

👏👏Създадохте полиномен модел, нека направим прогнози върху тестовия набор!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Ура, нека оценим как моделът се представи на test_set, използвайки `yardstick::metrics()`.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Много по-добра производителност.

`rmse` намаля от около 7 до около 3, което показва намалена грешка между реалната цена и предсказаната цена. Можете *грубо* да интерпретирате това като средно, че грешните предсказания се отклоняват с около 3 долара. `rsq` се увеличи от около 0.4 до 0.8.

Всички тези метрики показват, че полиномният модел се представя много по-добре от линейния модел. Отлична работа!

Нека видим дали можем да го визуализираме!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


Можете да видите извита линия, която по-добре пасва на вашите данни! 🤩

Можете да направите това още по-гладко, като подадете полиномиална формула на `geom_smooth`, ето така:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

Много прилича на гладка крива!🤩

Ето как можете да направите нова прогноза:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


Прогнозата на `полиномния модел` има смисъл, като се вземат предвид разпръснатите графики на `цена` и `пакет`! И ако това е по-добър модел от предишния, гледайки същите данни, трябва да планирате бюджет за тези по-скъпи тикви!

🏆 Браво! Създадохте два регресионни модела в един урок. В последната секция за регресия ще научите за логистичната регресия, за да определяте категории.

## **🚀Предизвикателство**

Тествайте няколко различни променливи в този тетрадка, за да видите как корелацията съответства на точността на модела.

## [**Тест след лекцията**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Преглед и Самостоятелно обучение**

В този урок научихме за Линейна регресия. Съществуват и други важни видове регресия. Прочетете за техниките Stepwise, Ridge, Lasso и Elasticnet. Един добър курс за изучаване на повече е [Статистическо обучение на Станфорд](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

Ако искате да научите повече за използването на невероятната рамка Tidymodels, моля, разгледайте следните ресурси:

-   Уебсайт на Tidymodels: [Започнете с Tidymodels](https://www.tidymodels.org/start/)

-   Макс Кун и Джулия Силге, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **БЛАГОДАРНОСТИ КЪМ:**

[Алисън Хорст](https://twitter.com/allison_horst?lang=en) за създаването на невероятните илюстрации, които правят R по-приветлив и ангажиращ. Намерете повече илюстрации в нейния [галерия](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия изходен език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за каквито и да е недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
